# **GC1**

Il modello GC1 ha la seguente struttura:

#### conv1: First convolutional layer (dim -> 256x256x8)
#### pool: First max pooling layer (dim -> 128x128x8)
#### conv2: Second convolutional layer (dim -> 128x128x16)
#### pool2: Second max pooling layer (dim -> 64x64x16)
#### conv3: Third convolutional layer (dim -> 64x64x32)
#### pool3: Third max pooling layer (dim -> 32x32x32)
#### fc1: Fully connected layer (dim -> num_classes)
#### softmax: Softmax layer (dim -> num_classes)

### **Problemi**

Overfitting: sale sul train ma sul validation si ferma a circa 65%

### **Possibili soluzioni**

- Aggiungere Dropout tra i layer completamente connessi
- Utilizzare tecniche di regularization come L2 regularization
- Sperimentare con diverse architetture di rete, come l'aggiunta di più layer convoluzionali o l'uso di architetture pre-addestrate (transfer learning)

# **GC2**

Il modello GC2 ha la seguente struttura:

#### conv1: First convolutional layer (dim -> 256x256x12)
#### conv2: Second convolutional layer (stride=2, kernel=5) (dim -> 126x126x16)
#### pool: First max pooling layer (dim -> 63x63x16)
#### conv3: Third convolutional layer (dim -> 63x63x24)
#### conv4: Fourth convolutional layer (stride=2, kernel=5) (dim -> 30x30x32)
#### pool2: Second max pooling layer (dim -> 15x15x32)
#### fc1: Fully connected layer (dim -> 1024)
#### dropout: Dropout layer (p=0.2)
#### fc2: Output layer (dim -> num_classes)

### **Modifiche rispetto a GC1**
- Aumentato il numero di filtri iniziali (8→12) e aggiunto un quarto layer convoluzionale per maggiore capacità espressiva.
- Riduzione della risoluzione tramite convoluzioni con stride=2 invece di soli max pooling, così da imparare anche la downsampling function.
- Inserito layer fully-connected intermedio a 1024 neuroni prima dell'output finale (GC1 andava direttamente a num_classes).
- Aggiunto dropout (p=0.2) tra i fully-connected per mitigare l'overfitting.
- Output ora su fc2 dedicato invece di un unico fc finale.

### **Problemi**
- Train accuracy: 100%
- Validation accuracy: 71.56%

### **Soluzioni**

Per ridurre l'eccessivo overfitting e migliorare la generalizzazione del modello GC2, si potrebbero adottare le seguenti strategie:
- **Aumentare il dropout**: Incrementare il tasso di dropout (ad esempio a 0.5) per ridurre ulteriormente la dipendenza del modello da specifici neuroni durante l'addestramento.
- **Batch normalization**: Integrare layer di batch normalization dopo le convoluzioni per stabilizzare e accelerare l'addestramento.
- **Kernel 3x3**: Sostituire i kernel 5x5 con kernel 3x3 per ridurre il numero di parametri e migliorare l'efficienza computazionale grazie allo stesso receptive field.
- **Aumento della Profondità Convoluzionale**: Piuttosto che layer lineari enormi, più layer convoluzionali per estrarre feature di alto livello.
- **Cambiare regolarizzazione**: Rimuovere la l1 regularization e sfruttare solo l2 dell'elastic net, per non perdere informazioni importanti portando a zero i pesi.

(l2 = 0.001, l1 = 0.000)